In [1]:
import pandas as pd
import pymysql
from datetime import datetime
from datetime import timedelta
import re

class MarketDB :
    def __init__(self):
        self.conn = pymysql.connect(host='localhost', user='root',
            password='qor100', db='INVESTAR', charset='utf8')
        self.codes = {}
        self.update_comp_info()
                            
    def __del__(self):
        self.conn.close() 
        
    def update_comp_info(self):
        sql = "SELECT * FROM company_info"
        df = pd.read_sql(sql, self.conn)
        for idx in range(len(df)):
            self.codes[df['code'].values[idx]] = df['company'].values[idx]
                    
    def get_daily_price(self, code, start_date=None, end_date=None):
        """시작 날짜 입력 내용"""
        if start_date is None:
            one_year_ago = datetime.today() - timedelta(days=365)
            start_date = one_year_ago.strftime('%Y-%m-%d')
            print("시작일을 입력하지 않아 작년부터 시작합니다. '{}'".format(start_date))
        else :
            start_lst = re.split('\D+', start_date) # 년, 월, 일 나눠서 리스트로 출력
            start_year = int(start_lst[0])
            start_month = int(start_lst[1])
            start_day = int(start_lst[2])
                
            if start_year < 1900 or start_year > 2200:
                print(f'({start_year:d}) 2017년도 부터 입력 가능합니다.')
                return
            if start_month < 1 or start_month > 12:
                print(f'({start_month:d}) 1~12월까지만 입력이 가능합니다.')
                return
            if start_day < 1 or start_day > 31 :
                print(f'({start_day:d}) 1~31일까지만 입력이 가능합니다.')
                return
            start_date=f"{start_year:04d}-{start_month:02d}-{start_day:02d}"
                
        if (end_date is None):
                end_date = datetime.today().strftime('%Y-%m-%d')
                print("마지막 일을 입력하지 않아 오늘 날짜로 시작합니다. '{}'".format(end_date))
        else :
                
            end_lst = re.split('\D+', end_date)
            end_year = int(end_lst[0])
            end_month = int(end_lst[1])
            end_day = int(end_lst[2])
            
            if end_year < 1900 or end_year > 2200:
                print(f'({end_year:d}) 2021년도 이전만 가능합니다.')
                return
            if end_month < 1 or end_month > 12:
                print(f'({end_month:d}) 1~12월까지만 입력이 가능합니다.')
                return
            if end_day < 1 or end_day > 31 :
                print(f'({end_day:d}) 1~31일까지만 입력이 가능합니다.')
                return
            end_date=f"{end_year:04d}-{end_month:02d}-{end_day:02d}"        
        
        codes_keys = list(self.codes.keys())
        codes_values = list(self.codes.values())
        if code in codes_keys:
            pass
        elif code in codes_values:
            idx = codes_values.index(code)
            code = codes_keys[idx]
        else :
            print("({}) 이 코드 및 회사명은 존재하지 않습니다.".format(code))

        sql = f"SELECT * FROM daily_price WHERE (code = '{code}' and date >= '{start_date}' and date <= '{end_date}')"
        df = pd.read_sql(sql, self.conn)
        df.index = df['date']
        return df